# Sentiment Classification with FHE

This notebook tackles sentiment classification with Fully Homomorphic Encryption. Let's imagine some client (could be a user or a company) wants to predict whether a specific text (e.g., a tweet) contains positive, neutral or negative feedback using a cloud service provider without actually revealing the text during the process.

To do this, we use a machine learning model that can predict over encrypted data thanks to the Concrete ML library available on [GitHub](https://github.com/zama-ai/concrete-ml).

The data-set we use in this notebook can be found on [Kaggle](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment). 
 
We present two different ways to encode the text:
1. A basic **TF-IDF** approach, which essentially looks at how often a word appears in the text.
2. An advanced **transformer** embedding of the text using the Huggingface repository.

The main assumption of this notebook is that clients, who want to have their text analyzed in a privacy preserving manner, can encode the text using a predefined representation before encrypting the data. The FHE-friendly model is thus trained in the clear beforehand for the given task, here classification, over theses representations using a relevant training set.

In [1]:
# Import the required packages
import os
import time

import numpy
import pandas as pd
from sklearn.metrics import average_precision_score
from sklearn.model_selection import GridSearchCV, train_test_split

from concrete.ml.sklearn import XGBClassifier

/data/bz620/fyp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!wget -O Tweets.csv https://huggingface.co/datasets/osanseviero/twitter-airline-sentiment/resolve/main/Tweets.csv

--2024-05-24 19:35:19--  https://huggingface.co/datasets/osanseviero/twitter-airline-sentiment/resolve/main/Tweets.csv
Resolving huggingface.co (huggingface.co)... 2600:9000:2553:fc00:17:b174:6d00:93a1, 2600:9000:2553:c200:17:b174:6d00:93a1, 2600:9000:2553:b400:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:2553:fc00:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ce/37/ce37f07d6007921b5a0a814bd1cb03df4a2fa91f9631a025317f3a3e4acbe83c/ea94b23f41892b290dec3330bb8cf9cb6b8bc669eaae5f3a84c40f7b0de8f15e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Tweets.csv%3B+filename%3D%22Tweets.csv%22%3B&response-content-type=text%2Fcsv&Expires=1716834919&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjgzNDkxOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jZS8zNy9jZTM3ZjA3ZDYwMDc5MjFiNWEwYTgxNGJkMWNiM

In [3]:
# Download the data-sets
if not os.path.isfile("Tweets.csv"):
    raise ValueError("Please launch the `download_data.sh` script in order to get the data-sets.")

train = pd.read_csv("Tweets.csv", index_col=0)
text_X = train["text"]
y = train["airline_sentiment"]
y = y.replace(["negative", "neutral", "positive"], [0, 1, 2])

pos_ratio = y.value_counts()[2] / y.value_counts().sum()
neg_ratio = y.value_counts()[0] / y.value_counts().sum()
neutral_ratio = y.value_counts()[1] / y.value_counts().sum()
print(f"Proportion of positive examples: {round(pos_ratio * 100, 2)}%")
print(f"Proportion of negative examples: {round(neg_ratio * 100, 2)}%")
print(f"Proportion of neutral examples: {round(neutral_ratio * 100, 2)}%")

Proportion of positive examples: 16.14%
Proportion of negative examples: 62.69%
Proportion of neutral examples: 21.17%


In [4]:
# Split in train test
text_X_train, text_X_test, y_train, y_test = train_test_split(
    text_X, y, test_size=0.1, random_state=42
)

### 1. Text representation using TF-IDF

[Term Frequency-Inverse Document Frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (TF-IDF) also known as  is a numerical statistic that is used to compute the importance of a term in a document. The higher the TF-IDF score, the more important the term is to the document.

We compute it as follows:

$$ \mathsf{TF\textrm{-}IDF}(t,d,D) = \mathsf{TF}(t,d) * \mathsf{IDF}(t,D) $$

where: $\mathsf{TF}(t,d)$ is the term frequency of term $t$ in document $d$, $\mathsf{IDF}(t,D)$ is the inverse document frequency of term $t$ in document collection $D$.

Here we use the scikit-learn implementation of TF-IDF vectorizer.

In [5]:
# Let's first build a representation vector from the text
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=500, stop_words="english")
X_train = tfidf_vectorizer.fit_transform(text_X_train)
X_test = tfidf_vectorizer.transform(text_X_test)

# Make our train and test dense array
X_train = X_train.toarray()
X_test = X_test.toarray()

In [6]:
# Let's build our model
model = XGBClassifier()

# A gridsearch to find the best parameters
parameters = {
    "n_bits": [2, 3],
    "max_depth": [1],
    "n_estimators": [10, 30, 50],
    "n_jobs": [-1],
}

In [7]:
# Run the gridsearch
grid_search = GridSearchCV(model, parameters, cv=3, n_jobs=1, scoring="accuracy")
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=XGBClassifier(), n_jobs=1,
             param_grid={'max_depth': [1], 'n_bits': [2, 3],
                         'n_estimators': [10, 30, 50], 'n_jobs': [-1]},
             scoring='accuracy')

In [8]:
# Check the accuracy of the best model
print(f"Best score: {grid_search.best_score_}")

# Check best hyper-parameters
print(f"Best parameters: {grid_search.best_params_}")

# Extract best model
best_model = grid_search.best_estimator_

Best score: 0.705980570734669
Best parameters: {'max_depth': 1, 'n_bits': 3, 'n_estimators': 50, 'n_jobs': -1}


In [9]:
# Compute the average precision for each class
y_proba_test_tfidf = best_model.predict_proba(X_test)

# Compute accuracy
y_pred_test_tfidf = numpy.argmax(y_proba_test_tfidf, axis=1)
accuracy_tfidf = numpy.mean(y_pred_test_tfidf == y_test)
print(f"Accuracy: {accuracy_tfidf:.4f}")

y_pred_positive = y_proba_test_tfidf[:, 2]
y_pred_negative = y_proba_test_tfidf[:, 0]
y_pred_neutral = y_proba_test_tfidf[:, 1]

ap_positive_tfidf = average_precision_score((y_test == 2), y_pred_positive)
ap_negative_tfidf = average_precision_score((y_test == 0), y_pred_negative)
ap_neutral_tfidf = average_precision_score((y_test == 1), y_pred_neutral)

print(f"Average precision score for positive class: " f"{ap_positive_tfidf:.4f}")
print(f"Average precision score for negative class: " f"{ap_negative_tfidf:.4f}")
print(f"Average precision score for neutral class: " f"{ap_neutral_tfidf:.4f}")

Accuracy: 0.7117
Average precision score for positive class: 0.6418
Average precision score for negative class: 0.8721
Average precision score for neutral class: 0.4376


In [10]:
# Let's see what are the top predictions based on the probabilities in y_pred_test
print("5 most positive tweets (class 2):")
for i in range(5):
    print(text_X_test.iloc[y_proba_test_tfidf[:, 2].argsort()[-1 - i]])

print("-" * 100)

print("5 most negative tweets (class 0):")
for i in range(5):
    print(text_X_test.iloc[y_proba_test_tfidf[:, 0].argsort()[-1 - i]])

5 most positive tweets (class 2):
@SouthwestAir I continue to be amazed by the amazing customer service.  Thank you SWA!
@JetBlue Great Thank you, lets hope so! Could you please notify me if flight 2302 leaves JFK? Thank you again
@JetBlue I do! The best airline wifi ever. Thank you!
@JetBlue Awesome thanks! Thanks for the quick response. You guys ROCK! :)
@united sent a DM just now. Thanks I am incredibly happy the fast response I got via Twitter than via customer care. Thank you
----------------------------------------------------------------------------------------------------
5 most negative tweets (class 0):
@USAirways been on hold 2 hours for a Cancelled Flighted flight. I understand the delay. I don't understand you auto-reFlight Booking Problems me on TUESDAY. HELP!
@SouthwestAir  placed on hold for total of two hours today after flight was Cancelled Flightled. Online option not available. What to do?
@southwestair I've been on hold for 2 hours to reschedule my Cancelled Flight

In [11]:
# Compile the model to get the FHE inference engine
# (this may take a few minutes depending on the selected model)
start = time.perf_counter()
best_model.compile(X_train)
end = time.perf_counter()
print(f"Compilation time: {end - start:.4f} seconds")

# Let's write a custom example and predict in FHE
tested_tweet = ["AirFrance is awesome, almost as much as Zama!"]
X_tested_tweet = tfidf_vectorizer.transform(numpy.array(tested_tweet)).toarray()
clear_proba = best_model.predict_proba(X_tested_tweet)

# Now let's predict with FHE over a single tweet and print the time it takes
start = time.perf_counter()
decrypted_proba = best_model.predict_proba(X_tested_tweet, fhe="execute")
end = time.perf_counter()
print(f"FHE inference time: {end - start:.4f} seconds")

Compilation time: 10.6687 seconds
FHE inference time: 1.1390 seconds


In [12]:
print(f"Probabilities from the FHE inference: {decrypted_proba}")
print(f"Probabilities from the clear model: {clear_proba}")

Probabilities from the FHE inference: [[0.30244059 0.17506451 0.5224949 ]]
Probabilities from the clear model: [[0.30244059 0.17506451 0.5224949 ]]


To sum up, 
- We trained a XGBoost model over TF-IDF representation of the tweets and their respective sentiment class. 
- The grid search gives us a model that achieves around ~70% accuracy.
- Given the imbalance in the classes, we rather compute the average precision per class.

Now we will see how we can approach the problem by leveraging the transformers power.

### 2. A transformer approach to text representation

[**Transformers**](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)) are neural networks that are often trained to predict the next words to appear in a text (this is commonly called self-supervised learning). 

They are powerful tools for all kind of Natural Language Processing tasks but supporting a transformer model in FHE might not always be ideal as they are quite big models. However, we can still leverage their hidden representation for any text and feed it to a more FHE friendly machine learning model (in this notebook we will use XGBoost) for classification.

Here we will use the transformer model from the amazing [**Huggingface**](https://huggingface.co/) repository.

In [13]:
import torch
import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Add MPS (for macOS with Apple Silicon or AMD GPUs) support when error is fixed. For now, we
# observe a decrease in torch's top1 accuracy when using MPS devices
# FIXME: https://github.com/zama-ai/concrete-ml-internal/issues/3953
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the tokenizer (converts text to tokens)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

# Load the pre-trained model
transformer_model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment-latest"
)

config.json: 100%|██████████| 929/929 [00:00<00:00, 1.91MB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 48.4MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 6.12MB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 503kB/s]
pytorch_model.bin: 100%|██████████| 501M/501M [00:04<00:00, 113MB/s]  
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mode

In [14]:
# Let's first see what are the model performance by itself
list_text_X_test = text_X_test.tolist()

tokenized_text_X_test = tokenizer.batch_encode_plus(
    list_text_X_test, pad_to_max_length=True, return_tensors="pt"
)["input_ids"]

# Depending on the hardware used, the number of examples to be processed can be reduced
# Here we split the data into 100 examples per batch
tokenized_text_X_test_split = torch.split(tokenized_text_X_test, split_size_or_sections=50)
transformer_model = transformer_model.to(device)

outputs = []
for tokenized_x_test in tqdm.tqdm(tokenized_text_X_test_split):
    tokenized_x = tokenized_x_test.to(device)
    output_batch = transformer_model(tokenized_x)["logits"]
    output_batch = output_batch.detach().cpu().numpy()
    outputs.append(output_batch)

outputs = numpy.concatenate(outputs, axis=0)

100%|██████████| 30/30 [00:01<00:00, 15.97it/s]


In [15]:
# Let's see what the transformer model predicts
print(f"Predictions for the first 3 tweets:\n {outputs[:3]}")

Predictions for the first 3 tweets:
 [[-2.3807466  -0.61802155  2.9900753 ]
 [ 2.0166502   0.49380693 -2.8006458 ]
 [ 2.3892703   0.1344361  -2.6873815 ]]


In [16]:
# Compute the metrics for each class

# Compute accuracy
accuracy_transformer_only = numpy.mean(numpy.argmax(outputs, axis=1) == y_test)
print(f"Accuracy: {accuracy_transformer_only:.4f}")

y_pred_positive = outputs[:, 2]
y_pred_negative = outputs[:, 0]
y_pred_neutral = outputs[:, 1]

ap_positive_transformer_only = average_precision_score((y_test == 2), y_pred_positive)
ap_negative_transformer_only = average_precision_score((y_test == 0), y_pred_negative)
ap_neutral_transformer_only = average_precision_score((y_test == 1), y_pred_neutral)

print(f"Average precision score for positive class: " f"{ap_positive_transformer_only:.4f}")
print(f"Average precision score for negative class: " f"{ap_negative_transformer_only:.4f}")
print(f"Average precision score for neutral class: " f"{ap_neutral_transformer_only:.4f}")

Accuracy: 0.8053
Average precision score for positive class: 0.8548
Average precision score for negative class: 0.9548
Average precision score for neutral class: 0.6801


It looks like the transformer outperforms the model built on TF-IDF reprensentation.
Unfortunately, running a transformer that big in FHE would be highly inefficient. 

Let's see if we can leverage transformer representation and train an FHE model for the given classification task. 

In [17]:
# Function that transforms a list of texts to their representation
# learned by the transformer.


def text_to_tensor(
    list_text_X_train: list,
    transformer_model: AutoModelForSequenceClassification,
    tokenizer: AutoTokenizer,
    device: str,
) -> numpy.ndarray:
    # Tokenize each text in the list one by one
    tokenized_text_X_train_split = []
    for text_x_train in list_text_X_train:
        tokenized_text_X_train_split.append(tokenizer.encode(text_x_train, return_tensors="pt"))

    # Send the model to the device
    transformer_model = transformer_model.to(device)
    output_hidden_states_list = []

    for tokenized_x in tqdm.tqdm(tokenized_text_X_train_split):
        # Pass the tokens through the transformer model and get the hidden states
        # Only keep the last hidden layer state for now
        output_hidden_states = transformer_model(tokenized_x.to(device), output_hidden_states=True)[
            1
        ][-1]
        # Average over the tokens axis to get a representation at the text level.
        output_hidden_states = output_hidden_states.mean(dim=1)
        output_hidden_states = output_hidden_states.detach().cpu().numpy()
        output_hidden_states_list.append(output_hidden_states)

    return numpy.concatenate(output_hidden_states_list, axis=0)


# Let's vectorize the text using the transformer
list_text_X_train = text_X_train.tolist()
list_text_X_test = text_X_test.tolist()

X_train_transformer = text_to_tensor(list_text_X_train, transformer_model, tokenizer, device)
X_test_transformer = text_to_tensor(list_text_X_test, transformer_model, tokenizer, device)

100%|██████████| 1464/1464 [00:10<00:00, 136.42it/s]


In [18]:
# Now we have a representation for each tweet, we can train a model on these.
grid_search = GridSearchCV(model, parameters, cv=3, n_jobs=1, scoring="accuracy")
grid_search.fit(X_train_transformer, y_train)

GridSearchCV(cv=3, estimator=XGBClassifier(), n_jobs=1,
             param_grid={'max_depth': [1], 'n_bits': [2, 3],
                         'n_estimators': [10, 30, 50], 'n_jobs': [-1]},
             scoring='accuracy')

In [19]:
# Check the accuracy of the best model
print(f"Best score: {grid_search.best_score_}")

# Check best hyper-parameters
print(f"Best parameters: {grid_search.best_params_}")

# Extract best model
best_model = grid_search.best_estimator_

Best score: 0.8382665452337584
Best parameters: {'max_depth': 1, 'n_bits': 3, 'n_estimators': 50, 'n_jobs': -1}


In [20]:
# Compute the metrics for each class

y_proba = best_model.predict_proba(X_test_transformer)

# Compute the accuracy
y_pred = numpy.argmax(y_proba, axis=1)
accuracy_transformer_xgboost = numpy.mean(y_pred == y_test)
print(f"Accuracy: {accuracy_transformer_xgboost:.4f}")

y_pred_positive = y_proba[:, 2]
y_pred_negative = y_proba[:, 0]
y_pred_neutral = y_proba[:, 1]

ap_positive_transformer_xgboost = average_precision_score((y_test == 2), y_pred_positive)
ap_negative_transformer_xgboost = average_precision_score((y_test == 0), y_pred_negative)
ap_neutral_transformer_xgboost = average_precision_score((y_test == 1), y_pred_neutral)

print(f"Average precision score for positive class: " f"{ap_positive_transformer_xgboost:.4f}")
print(f"Average precision score for negative class: " f"{ap_negative_transformer_xgboost:.4f}")
print(f"Average precision score for neutral class: " f"{ap_neutral_transformer_xgboost:.4f}")

Accuracy: 0.8463
Average precision score for positive class: 0.8960
Average precision score for negative class: 0.9647
Average precision score for neutral class: 0.7449


Our FHE-friendly XGBoost model does 38% better than the XGBoost model built over TF-IDF representation of the text. Note that here we are still not using FHE and only evaluating the model.
Interestingly, using XGBoost over the transformer representation of the text matches the performance of the transformer model alone.

In [21]:
# Get probabilities predictions in clear
y_pred_test = best_model.predict_proba(X_test_transformer)

# Let's see what are the top predictions based on the probabilities in y_pred_test
print("5 most positive tweets (class 2):")
for i in range(5):
    print(text_X_test.iloc[y_pred_test[:, 2].argsort()[-1 - i]])

print("-" * 100)

print("5 most negative tweets (class 0):")
for i in range(5):
    print(text_X_test.iloc[y_pred_test[:, 0].argsort()[-1 - i]])

5 most positive tweets (class 2):
@united I think this is the best first class I have ever gotten!!  Denver to LAX and it's wonderful!!!
@AmericanAir Flight 236 was great. Fantastic cabin crew. A+ landing. #thankyou #JFK http://t.co/dRW08djHAI
@SouthwestAir Jason (108639) at Gate #3 in SAN made my afternoon!!! #southwestairlines #stellarservice #thanks!
@SouthwestAir love them! Always get the best deals!
@AmericanAir simply amazing. Smiles for miles.Thank u for my upgrade tomorrow for ORD.We are spending a lot of time together next few weeks!
----------------------------------------------------------------------------------------------------
5 most negative tweets (class 0):
@united first you lost all my bags, now you Cancelled Flight my flight home. 30 min wait to talk to somebody #poorservice #notgoodenough
@USAirways Not only did u lose the flight plan! Now ur flight crew is FAA timed out! Thx for havin us sit on the tarmac for an hr! #Pathetic
@AmericanAir Phone just disconnects if

In [22]:
# Now let's see where the model is wrong
y_pred_test_0 = y_pred_test[y_test == 0]
text_X_test_0 = text_X_test[y_test == 0]

print("5 most positive (predicted) tweets that are actually negative (ground truth class 0):")
for i in range(5):
    print(text_X_test_0.iloc[y_pred_test_0[:, 2].argsort()[-1 - i]])

print("-" * 100)

y_pred_test_2 = y_pred_test[y_test == 2]
text_X_test_2 = text_X_test[y_test == 2]
print("5 most negative (predicted) tweets that are actually positive (ground truth class 2):")
for i in range(5):
    print(text_X_test_2.iloc[y_pred_test_2[:, 0].argsort()[-1 - i]])

5 most positive (predicted) tweets that are actually negative (ground truth class 0):
@united thanks for the link, now finally arrived in Brussels, 9 h after schedule...
@USAirways as far as being delayed goes… Looks like tailwinds are going to make up for it. Good news!
@united thanks for having changed me. Managed to arrive with only 8 hours of delay and exhausted
@USAirways your saving grace was our flight attendant Dallas who was amazing. wish he would transfer to Delta where I would see him again
@AmericanAir that luggage you forgot...#mia.....he just won an oscar😄💝💝💝
----------------------------------------------------------------------------------------------------
5 most negative (predicted) tweets that are actually positive (ground truth class 2):
@united thanks for updating me about the 1+ hour delay the exact second I got to ATL. 🙅🙅🙅
@SouthwestAir save mile to visit family in 2015 and this will impact how many times I can see my mother.  I planned and you change the rules
@J

Interestingly, these misclassifications are not obvious and some actually look rather like mislabeled. Also, it seems that the model is having a hard time to find ironic tweets.

Now we have our model trained which has some great accuracy. Let's have it predict over the encrypted representation.

### Sentiment Analysis of the Tweet with Fully Homomorphic Encryption

Now that we have our model ready for FHE inference and our data ready for encryption let's use the model in a privacy preserving manner with FHE.

In [23]:
# Compile the model to get the FHE inference engine
# (this may take a few minutes depending on the selected model)
start = time.perf_counter()
best_model.compile(X_train_transformer)
end = time.perf_counter()
print(f"Compilation time: {end - start:.4f} seconds")


# Let's write a custom example and predict in FHE
tested_tweet = ["AirFrance is awesome, almost as much as Zama!"]
X_tested_tweet = text_to_tensor(tested_tweet, transformer_model, tokenizer, device)
clear_proba = best_model.predict_proba(X_tested_tweet)

# Now let's predict with FHE over a single tweet and print the time it takes
start = time.perf_counter()
decrypted_proba = best_model.predict_proba(X_tested_tweet, fhe="execute")
end = time.perf_counter()
fhe_exec_time = end - start
print(f"FHE inference time: {fhe_exec_time:.4f} seconds")

Compilation time: 12.4545 seconds


100%|██████████| 1/1 [00:00<00:00, 33.87it/s]


FHE inference time: 0.9529 seconds


In [24]:
print(f"Probabilities from the FHE inference: {decrypted_proba}")
print(f"Probabilities from the clear model: {clear_proba}")

Probabilities from the FHE inference: [[0.05162184 0.04558276 0.90279541]]
Probabilities from the clear model: [[0.05162184 0.04558276 0.90279541]]


In [25]:
# Let's export the final model such that we can reuse it in a client/server environment

# Export some data to be used for compilation
X_train_numpy = X_train_transformer[:100]

# Merge the two arrays in a pandas dataframe
X_test_numpy_df = pd.DataFrame(X_train_numpy)

# to csv
X_test_numpy_df.to_csv("samples_for_compilation.csv")

# Let's save the model to be pushed to a server later
from concrete.ml.deployment import FHEModelDev

fhe_api = FHEModelDev("sentiment_fhe_model", best_model)
fhe_api.save()

In [26]:
%matplotlib inline
# Let's print the results obtained in this notebook
df_results = pd.DataFrame(
    {
        "Model": ["TF-IDF + XGBoost", "Transformer Only", "Transformer + XGBoost"],
        "Accuracy": [accuracy_tfidf, accuracy_transformer_only, accuracy_transformer_xgboost],
        "Average Precision (positive)": [
            ap_positive_tfidf,
            ap_positive_transformer_only,
            ap_positive_transformer_xgboost,
        ],
        "Average Precision (negative)": [
            ap_negative_tfidf,
            ap_negative_transformer_only,
            ap_negative_transformer_xgboost,
        ],
        "Average Precision (neutral)": [
            ap_neutral_tfidf,
            ap_neutral_transformer_only,
            ap_neutral_transformer_xgboost,
        ],
    }
)
df_results.set_index("Model", inplace=True)
df_results  # pylint: disable=pointless-statement

ModuleNotFoundError: No module named 'matplotlib'

### Conclusion

In this notebook we presented two different ways to represent a text.
1. Using TF-IDF vectorization
2. Using the hidden layers from a transformer

Both representation are then used to train a machine learning model will run in FHE (here XGBoost)

Once the model is trained, clients can send encrypted text representation to the server to get a sentiment analysis done and they receive the probability for each class (negative, neutral and positive) in an encrypted format which can then be decrypted by the client. For now, all the FHE magic (encrypt, predict and decrypt) is done within the `predict_proba` function with the argument `fhe='execute'`. In the next release, an API will be provided to split the server/client parts.

Regarding the FHE execution times, the final XGboost model can predict over an encrypted data point in ~4 seconds. This will change depending on the number of threads available. In the future, more hardware acceleration will be available to speed up the execution time.

It seems that the combination of a transformer (thanks Huggingface!) with a "simpler" model such as XGBoost works pretty well. Thanks to Concrete ML library, we can easily use this text representation on the client machine and then encrypt it to send it to a remote server without having to deal with a transformer runtime in FHE.